In [1]:
'''Packages'''
import re
import codecs
import pandas as pd
from nltk.tokenize import TreebankWordTokenizer, WhitespaceTokenizer
from nltk.corpus import stopwords
import numpy as np
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer
from heapq import nlargest
from operator import itemgetter
from collections import Counter
from nltk import tokenize
from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize
import spacy
from bs4 import BeautifulSoup
nltk.download('stopwords')
import warnings
warnings.filterwarnings("ignore")
from sklearn.decomposition import NMF
from gensim.models.coherencemodel import CoherenceModel


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [91]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [2]:
import nltk
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

#  Preprocess dataset with the functions from the Part 1

In [3]:
data_optional = pd.read_csv("data_optional.csv")

In [4]:
#remove all except words
data_optional['text_original'] =  data_optional['text']
for i in range(len(data_optional)):
    data_optional['text'][i] = re.sub('([^A-Za-z0-9 ]|[^ ]*[0-9][^ ]*)', ' ', data_optional['text'][i]).lower()

In [5]:
#tokenizer
array_for_tokens = []
for i in range(len(data_optional)):
    array_for_tokens.append(TreebankWordTokenizer().tokenize(data_optional["text"][i]))


data_optional['tokens'] = array_for_tokens

#stopwords 
from nltk.corpus import stopwords
stop_words = stopwords.words("english")


lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()


clean = []
stop_my_words = ['com','u', 'inc', 'msn']
for i in range(len(data_optional)):
    #deleting stopwords
    tokens_ns = [token for token in data_optional["tokens"][i] if token not in stop_words]
    tokens_ns2 = [token for token in tokens_ns if token not in stop_my_words]
    #lemmatization
    clean.append([lemmatizer.lemmatize(token) for token in tokens_ns2])



data_optional['tokens_ns'] = clean

data_optional["text_cleaned"] = [' '.join(i) for i in data_optional["tokens_ns"]]

data_optional.head()

,id,text,author,text_original,tokens,tokens_ns,text_cleaned
0,id26305,this process however afforded me no means of...,EAP,"This process, however, afforded me no means of...","[this, process, however, afforded, me, no, mea...","[process, however, afforded, mean, ascertainin...",process however afforded mean ascertaining dim...
1,id17569,it never once occurred to me that the fumbling...,HPL,It never once occurred to me that the fumbling...,"[it, never, once, occurred, to, me, that, the,...","[never, occurred, fumbling, might, mere, mistake]",never occurred fumbling might mere mistake
2,id11008,in his left hand was a gold snuff box from wh...,EAP,"In his left hand was a gold snuff box, from wh...","[in, his, left, hand, was, a, gold, snuff, box...","[left, hand, gold, snuff, box, capered, hill, ...",left hand gold snuff box capered hill cutting ...
3,id27763,how lovely is spring as we looked from windsor...,MWS,How lovely is spring As we looked from Windsor...,"[how, lovely, is, spring, as, we, looked, from...","[lovely, spring, looked, windsor, terrace, six...",lovely spring looked windsor terrace sixteen f...
4,id12958,finding nothing else not even gold the super...,HPL,"Finding nothing else, not even gold, the Super...","[finding, nothing, else, not, even, gold, the,...","[finding, nothing, else, even, gold, superinte...",finding nothing else even gold superintendent ...


In [6]:
data_train = data_optional

In [7]:
#train
flat_list_train = [item for sublist in data_train['tokens_ns'] for item in sublist]
tokens_train =  Counter(flat_list_train)
tokens_count_df_train =  pd.DataFrame.from_dict(tokens_train, orient='index').reset_index()
tokens_count_df_train.columns = ['word','count']
tokens_count_df_train = tokens_count_df_train.sort_values(by=['count'],ascending=False)
import plotly.express as px
fig = px.bar(tokens_count_df_train[0:100], x='word', y='count',title="Words histogram train")
fig.show()

#Topic modeling

##NMF

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(
    preprocessor=''.join
)
tfidf = tfidf_vectorizer.fit_transform(data_train['text_cleaned'].tolist())


In [9]:
model = NMF(n_components=5, random_state=10)

model.fit(tfidf)
nmf_features = model.transform(tfidf)
components_df = pd.DataFrame(model.components_, columns=tfidf_vectorizer.get_feature_names())


In [100]:
topic_NMF = []
words_NMF = []
for topic in range(components_df.shape[0]):
    tmp = components_df.iloc[topic]
    words_NMF.append(components_df.iloc[topic].nlargest(10).index.tolist())
    topic_NMF.append(topic)
NMF_result = pd.DataFrame()
NMF_result['topic'] = topic_NMF
NMF_result['words'] = words_NMF

In [106]:
NMF_result

,topic,words
0,0,"[upon, time, man, day, thing, yet, eye, first,..."
1,1,"[one, another, two, thing, old, every, seen, s..."
2,2,"[said, know, dupin, let, nothing, little, sir,..."
3,3,"[could, nothing, see, yet, word, make, perceiv..."
4,4,"[would, return, make, well, might, knew, thoug..."


##LDA

In [107]:
import gensim
from gensim import corpora

In [108]:
dictionary = gensim.corpora.Dictionary(data_train['tokens_ns'])

corpus = [dictionary.doc2bow(text) for text in data_train['tokens_ns']]

In [109]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [110]:
# Save 10 words for topics
lda_model.show_topics(formatted = False,num_words=10)
doc_lda = lda_model[corpus]

In [111]:
words = []
for i in range(len(lda_model.show_topics(formatted = False,num_words=10))):
    temp = []
    for j in range(len(lda_model.show_topics(formatted = False,num_words=10)[i][1])): 
        temp.append(lda_model.show_topics(formatted = False,num_words=10)[i][1][j][0])
    words.append(temp)

In [112]:
LDA_result = pd.DataFrame()
LDA_result['topic'] = range(5)
LDA_result['words'] = words

In [113]:
LDA_result

,topic,words
0,0,"[would, man, even, life, never, old, must, gre..."
1,1,"[one, upon, could, time, day, thing, found, ni..."
2,2,"[like, eye, saw, seemed, still, water, though,..."
3,3,"[made, first, hand, voice, word, come, idea, h..."
4,4,"[yet, said, may, u, little, say, love, much, w..."


##ARTM

In [60]:
flat_list = [item for sublist in data_train['tokens_ns'] for item in sublist]


In [21]:
full_text = ' '.join(flat_list)

In [22]:
!pip install bigartm

     |████████████████████████████████| 1.9 MB 28.0 MB/s 


In [23]:
import artm

# Prepare data
# Case 1: data in CountVectorizer format
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.datasets import fetch_20newsgroups
from numpy import array



In [24]:
cv = CountVectorizer(max_features=1000, stop_words='english')
n_wd = array(cv.fit_transform(data_train['text_cleaned'].tolist()).todense()).T
vocabulary = cv.get_feature_names()

In [25]:
bv = artm.BatchVectorizer(data_format='bow_n_wd',
                          n_wd=n_wd,
                          vocabulary=vocabulary)


model = artm.LDA(num_topics=5, dictionary=bv.dictionary)
model.fit_offline(bv, num_collection_passes=20)

In [27]:
ARTM_result = pd.DataFrame()
ARTM_result['topic'] = range(5)
ARTM_result['words'] = model.get_top_tokens(10)

In [28]:
ARTM_result

,topic,words
0,0,"[night, mind, heart, friend, heard, felt, love..."
1,1,"[day, eye, little, house, year, word, left, he..."
2,2,"[long, came, time, earth, father, city, foot, ..."
3,3,"[man, life, death, place, far, shall, men, com..."
4,4,"[thing, said, thought, say, old, know, like, l..."


# Coherence calculation

In [72]:
from gensim.test.utils import common_corpus, common_dictionary
from gensim.models.coherencemodel import CoherenceModel

In [73]:
dictionary = gensim.corpora.Dictionary(data_optional["tokens_ns"])
corpus = [dictionary.doc2bow(text) for text in data_optional["tokens_ns"]]

In [74]:
cm = CoherenceModel(topics=ARTM_result['words'], corpus=corpus, dictionary=dictionary, coherence='u_mass')
coherence = cm.get_coherence()
print("ARTM coherence = " ,np.exp(coherence))

ARTM coherence =  0.020398418455759886


In [75]:
cm = CoherenceModel(topics=LDA_result['words'], corpus=corpus, dictionary=dictionary, coherence='u_mass')
coherence = cm.get_coherence()
print("LDA coherence = ", np.exp(coherence))

LDA coherence =  0.029707094117831945


In [76]:
cm = CoherenceModel(topics=NMF_result['words'], corpus=corpus, dictionary=dictionary, coherence='u_mass')
coherence = cm.get_coherence()
print("NMF coherence = ",np.exp(coherence))

NMF coherence =  0.010767014359492537


# NMPI Calculation

In [79]:
def get_pairs(items):
  a = [(items[i],items[j]) for i in range(len(items)) for j in range(i+1, len(items))]
  return a

def NMPI_calculate(topic_df):
  NMPI_list = []
  for topic in range(len(topic_df)):
    items = topic_df['words'][topic]
    NMPI = 0
    for pair in get_pairs(items):
      pi = 0
      pj = 0
      pij = 0
      for document in data_train['tokens_ns']:
        tempi = False
        tempj = False
        if pair[0] in document:
          pi+=1
          tempi = True
        if pair[1] in document:
          pj+=1
          tempj = True
        if tempi and tempj:
          pij += 1
      pi = pi/len(data_train['tokens_ns'])
      pj = pj/len(data_train['tokens_ns'])
      if pij != 0:
        pij = pij/len(data_train['tokens_ns'])
      else:
        continue
      NMPI += np.log(pij/(pi*pj))/(-np.log(pij))
    NMPI_list.append(NMPI)
  return NMPI_list
  

In [83]:
NMPI_ARTM = NMPI_calculate(ARTM_result)
print('ARTM NMPI per topic: ', NMPI_ARTM) 
print('ARTM NMPI mean : ', np.mean(NMPI_ARTM))

ARTM NMPI per topic:  [2.060906771430453, 0.7413235851871691, 2.489739957063488, 2.117183260411967, 2.0651440690990093]
ARTM NMPI mean :  1.8948595286384173


In [85]:
NMPI_LDA = NMPI_calculate(LDA_result)
print('LDA NMPI per topic: ', NMPI_LDA) 
print('LDA NMPI mean : ', np.mean(NMPI_LDA))

LDA NMPI per topic:  [1.8930759068070069, 2.6572190016742816, 2.0243985448776947, 1.1554349914843478, 2.8552219119289797]
LDA NMPI mean :  2.117070071354462


In [86]:
NMPI_NMF = NMPI_calculate(NMF_result)
print('NMF NMPI per topic: ', NMPI_NMF) 
print('NMF NMPI mean : ', np.mean(NMPI_NMF))

NMF NMPI per topic:  [1.5912385856881293, 2.580736322249636, 4.1617165369011095, 3.246669848006724, 3.139450390469324]
NMF NMPI mean :  2.9439623366629846


# Based on distributed representation


In [71]:
import gensim.downloader
glove_vectors = gensim.downloader.load('glove-twitter-25')
from gensim.models import Word2Vec

/usr/local/lib/python3.7/dist-packages/smart_open/smart_open_lib.py:494: DeprecationWarning:

This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information



In [89]:
def disturbed_calculate(topic_df):
  per_topic = []
  for topic in range(len(topic_df)):
    items = topic_df['words'][topic]
    temp_sim = []
    for pair in get_pairs(items):
      try:
        temp_sim.append(glove_vectors.similarity(pair[0], pair[1]))
      except KeyError:
        continue
    per_topic.append(np.mean(temp_sim))

  return per_topic

In [96]:
dis_ARTM = disturbed_calculate(ARTM_result)
print('ARTM distributed representation per topic: ', dis_ARTM) 
print('ARTM distributed representation mean : ', np.mean(dis_ARTM))

ARTM distributed representation per topic:  [0.7088382, 0.6306983, 0.7032531, 0.7788851, 0.79795945]
ARTM distributed representation mean :  0.72392684


In [97]:
dis_LDA = disturbed_calculate(LDA_result)
print('LDA distributed representation per topic: ', dis_LDA) 
print('LDA distributed representation mean : ', np.mean(dis_LDA))

LDA distributed representation per topic:  [0.8435468, 0.8322114, 0.7525609, 0.71602875, 0.80553275]
LDA distributed representation mean :  0.78997606


In [99]:
dis_NMF = disturbed_calculate(NMF_result)
print('NMF distributed representation per topic: ', dis_NMF) 
print('NMF distributed representation mean : ', np.mean(dis_NMF))

NMF distributed representation per topic:  [0.80515206, 0.8896055, 0.8140967, 0.7245761, 0.89320207]
NMF distributed representation mean :  0.82532656
